In [1]:
from pathlib import Path
import dfm_tools as dfmt

import geopandas
import xarray as xr
import xugrid as xu
import pandas as pd
from shapely.geometry import Point

In [4]:
## create gdf:  
# https://github.com/Deltares/dfm_tools/blob/5d3bbad65b083f08a2f6e9a73bb60d3e21f19231/dfm_tools/hydrolib_helpers.py#L371
# above link = not really right format for here...

def dutch_crs_plipoint_list_to_decimaldeg_gdf(list_plifiles):
    '''
    input:
    - list of all the plifiles as WindowsPath. Format = from offshore boundary pli files. Dutch crs!
    
    output:
    - gdf of all the pli files with columns: location, geometry (Points). Crs = 4326 (decimal degrees)
    
    '''
    all_boundaries = []
    for i,file in list(enumerate(list_plifiles)):  # start with zuid (as did for testing)
        print(str(file).split('\\')[-1].split('.')[0])

        # Read points within this offshore boundary:
        pli_file = pd.read_csv(list_plifiles[i], skiprows=1, sep=' ')   # read pli file
        pli_file = pli_file.dropna(axis=1)                              # drop empty columns
        # Convert dutch crs to decimal degrees (for DCSM)
        points = [] 
        for i in range(0,len(pli_file)):
            points.append(Point(pli_file[pli_file.columns[0]].to_list()[i], pli_file[pli_file.columns[1]].to_list()[i]))
        d = {'location': pli_file[pli_file.columns[2]].to_list(), 'geometry': points}
        gdf = geopandas.GeoDataFrame(d, crs='epsg:28992')   # set Dutch crs
        gdf = gdf.to_crs(4326)                              # convert to decimal degrees
        all_boundaries.append(gdf)
    all_boundaries = pd.concat(all_boundaries, ignore_index=True)
    return all_boundaries



# Read the pli points:
list_plifiles = [Path(r'p:\11208479-sequestration-seaweed\Oosterschelde_DFM_hydro_waq\dflowfm3d-oosterschelde-wq\boundary_conditions\zee-noord.pli'),
                Path(r'p:\11208479-sequestration-seaweed\Oosterschelde_DFM_hydro_waq\dflowfm3d-oosterschelde-wq\boundary_conditions\zee-west.pli'),
                Path(r'p:\11208479-sequestration-seaweed\Oosterschelde_DFM_hydro_waq\dflowfm3d-oosterschelde-wq\boundary_conditions\zee-zuid.pli')]
# Use function
all_boundaries = dutch_crs_plipoint_list_to_decimaldeg_gdf(list_plifiles)
all_boundaries

zee-noord
zee-west
zee-zuid


,location,geometry
0,zee-noord_0001,POINT (3.68494 51.74581)
1,zee-noord_0002,POINT (3.68651 51.74360)
2,zee-noord_0003,POINT (3.69331 51.73290)
3,zee-noord_0004,POINT (3.69707 51.72693)
4,zee-noord_0005,POINT (3.69942 51.72311)
5,zee-west_0001,POINT (3.43000 51.60113)
6,zee-west_0002,POINT (3.44703 51.61624)
7,zee-west_0003,POINT (3.47948 51.64333)
8,zee-west_0004,POINT (3.51249 51.66873)
9,zee-west_0005,POINT (3.55542 51.69705)


In [18]:
## extracting offshore pli points from DFM:

def extracting_plipoints_from_DFM(gdf, point_index, model_output):
    '''
    input:
    - gdf with location, geometry (Point) and crs=4326
    - point_index: Index number or False
    - model_output: dfm model output read using dfm_tools. 
           Dims: mesh2d_nLayers, mesh2d_nInterfaces, time, mesh2d_nNodes, mesh2d_nFaces, mesh2d_nMax_face_nodes, mesh2d_nEdges
    
    output:
    - xr.Dataset with dims: plipoints, time, depth
    
    '''

    list_point_ds = []
    ### Version for when select only a point:
    for point in range(0,len(gdf))[point_index:(point_index+1)]:        # loop over points
        depths_slice = model_output.mesh2d_layer_z.values
        # convert depths (sigma to meter):
        ds_atdepths = dfmt.get_Dataset_atdepths(data_xr=model_output, depths=depths_slice)
        # select pli point
        ds_atdepths_sel = ds_atdepths.ugrid.sel(x=gdf.geometry[point].x, y=gdf.geometry[point].y)
        # rename the variables
        ds_atdepths_sel = dfmt.rename_waqvars(ds_atdepths_sel)
        ds_atdepths_sel = ds_atdepths_sel.rename({'depth_from_z0':'depth'})
        ds_atdepths_sel = ds_atdepths_sel.rename({'mesh2d_nFaces':'plipoints'}) # rename mesh2d_nFaces to plipoints
        ds_atdepths_sel['plipoints'] = xr.DataArray([gdf.location[point]], dims='plipoints') # change name of plipoint (node to gdf name)
        list_point_ds.append(ds_atdepths_sel)

        ### Version when want full ds:
#         for point in range(0,len(gdf)):
#             depths_slice = model_output.mesh2d_layer_z.values
#             # convert depths (sigma to meter):
#             ds_atdepths = dfmt.get_Dataset_atdepths(data_xr=model_output, depths=depths_slice)
#             # select pli point
#             ds_atdepths_sel = ds_atdepths.ugrid.sel(x=gdf.geometry[point].x, y=gdf.geometry[point].y)
#             # rename the variables
#             ds_atdepths_sel = dfmt.rename_waqvars(ds_atdepths_sel)
#             ds_atdepths_sel = ds_atdepths_sel.rename({'depth_from_z0':'depth'})
#             ds_atdepths_sel = ds_atdepths_sel.rename({'mesh2d_nFaces':'plipoints'}) # rename mesh2d_nFaces to plipoints
#             ds_atdepths_sel['plipoints'] = xr.DataArray([gdf.location[point]], dims='plipoints') # change name of plipoint (node to gdf name)
#             list_point_ds.append(ds_atdepths_sel)
    
    ds = xr.concat(list_point_ds, dim='plipoints')
    return ds


# read model (multi-years)
chunks = {'time':1}
files_p1 = [r'p:\\archivedprojects\\11206044-002ospareutrophication\\final_results_Snellius_latestversion_20220210\\A16b_Numtopsiguniform1_2015_pH\\DFM_OUTPUT_DCSM-FM_0_5nm_waq\\DCSM-FM_0_5nm_waq_0146_map.nc',r'p:\\archivedprojects\\11206044-002ospareutrophication\\final_results_Snellius_latestversion_20220210\\A16b_Numtopsiguniform1_2016_pH\\DFM_OUTPUT_DCSM-FM_0_5nm_waq\\DCSM-FM_0_5nm_waq_0146_map.nc']
files_p2 = [r'p:\\archivedprojects\\11206044-002ospareutrophication\\final_results_Snellius_latestversion_20220210\\A16b_Numtopsiguniform1_2015_pH\\DFM_OUTPUT_DCSM-FM_0_5nm_waq\\DCSM-FM_0_5nm_waq_0147_map.nc',r'p:\\archivedprojects\\11206044-002ospareutrophication\\final_results_Snellius_latestversion_20220210\\A16b_Numtopsiguniform1_2016_pH\\DFM_OUTPUT_DCSM-FM_0_5nm_waq\\DCSM-FM_0_5nm_waq_0147_map.nc']
files_p3 = [r'p:\\archivedprojects\\11206044-002ospareutrophication\\final_results_Snellius_latestversion_20220210\\A16b_Numtopsiguniform1_2015_pH\\DFM_OUTPUT_DCSM-FM_0_5nm_waq\\DCSM-FM_0_5nm_waq_0148_map.nc',r'p:\\archivedprojects\\11206044-002ospareutrophication\\final_results_Snellius_latestversion_20220210\\A16b_Numtopsiguniform1_2016_pH\\DFM_OUTPUT_DCSM-FM_0_5nm_waq\\DCSM-FM_0_5nm_waq_0148_map.nc']
all_p1 = xu.open_mfdataset(files_p1,chunks=chunks)
all_p2 = xu.open_mfdataset(files_p2,chunks=chunks)
all_p3 = xu.open_mfdataset(files_p3,chunks=chunks)
all_xu = xu.merge_partitions([all_p1,all_p2, all_p3])

# use definition:
pli_selection = extracting_plipoints_from_DFM(gdf=all_boundaries, point_index=True, model_output=all_xu)

>> found layer/interface dimensions in file: mesh2d_nLayers mesh2d_nInterfaces
zw/zcc (fullgrid) values already present in Dataset in variable mesh2d_flowelem_zw
>> subsetting data on fixed depth in fullgrid z-data: 1.98 sec


In [19]:
## Rename any var with mesh_2d to tracerbnd:

def rename_dfm_wq_variables_to_bc_tracerbnd(ds):
    '''
    Imput: 
    - old dataset
    
    Output:
    - renamed ds, otherwise same format
    '''
    old_names = []
    for i in (ds.data_vars):
        old_names.append(i)

    new_names = []
    for i in list(ds.data_vars):    
        if 'mesh2d' in i:        
            i = i.replace('mesh2d_', 'tracerbnd')
            new_names.append(i)
        else:
            new_names.append(i)

    for i in range(0,len(old_names)):
        ds = ds.rename({old_names[i]: new_names[i]})
        
    return ds

rename_dfm_wq_variables_to_bc_tracerbnd(pli_selection)

<xarray.Dataset>
Dimensions:                           (plipoints: 1, time: 108, depth: 50)
Coordinates:
  * time                              (time) datetime64[ns] 2015-01-01 ... 20...
    mesh2d_face_x                     (plipoints) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    mesh2d_face_y                     (plipoints) float64 dask.array<chunksize=(1,), meta=np.ndarray>
  * plipoints                         (plipoints) <U14 'zee-noord_0002'
  * depth                             (depth) float64 -5.137e+03 ... -2.5
    x                                 (plipoints) float64 3.687
    y                                 (plipoints) float64 51.74
Data variables: (12/113)
    wgs84                             (plipoints) int32 -2147483647
    timestep                          (plipoints, time) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
    tracerbndflowelem_ba              (plipoints) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    tracerbndflowelem_bl              (plipoints) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    tracerbndflowelem_domain          (plipoints) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    tracerbndflowelem_globalnr        (plipoints) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...                                ...
    Limit sil                         (time, plipoints, depth) float64 dask.array<chunksize=(1, 1, 50), meta=np.ndarray>
    Limit e                           (time, plipoints, depth) float64 dask.array<chunksize=(1, 1, 50), meta=np.ndarray>
    Limit gro                         (time, plipoints, depth) float64 dask.array<chunksize=(1, 1, 50), meta=np.ndarray>
    Limit mor                         (time, plipoints, depth) float64 dask.array<chunksize=(1, 1, 50), meta=np.ndarray>
    FRACTIME01                        (time, plipoints, depth) float64 dask.array<chunksize=(1, 1, 50), meta=np.ndarray>
    tracerbndflowelem_zcc             (time, plipoints, depth) float64 dask.array<chunksize=(1, 1, 50), meta=np.ndarray>
Attributes:
    institution:    Deltares
    references:     http://www.deltares.nl
    source:         D-Flow FM 1.2.110.67924. Model:
    history:        Created on 2021-09-21T19:08:03+0200, D-Flow FM
    date_created:   2021-09-21T19:08:03+0200
    date_modified:  2021-09-21T19:08:03+0200
    Conventions:    CF-1.8 UGRID-1.0 Deltares-0.10

In [ ]:
# Loop over the variables and create .bc file:

## List the variables
list_quantities = ['tracerbndNO3', 'tracerbndOXY', 'tracerbndNH4', 'tracerbndPO4', 'tracerbndSi', 'tracerbndOpal', 
                   'tracerbndPOC1', 'tracerbndPON1', 'tracerbndPOP1'] #, 'DOC', 'DON', 'DOP']

for num,name in list(enumerate(list_quantities)):    # loop over variables
    ForcingModel_object = dfmt.plipointsDataset_to_ForcingModel(plipointsDataset=ds[[list_quantities[num]]])
    # Save and write the file
    try:
        name.split('tracerbnd')[-1]  # if the name has 'tracerbnd', then use last part
    except:
        pass
    ForcingModel_object.save(fr'...\{name}.bc')